# Data Preprocessing
- The **goal** is to load, process, analyze, feature engineer, and download an economic data set

In [ ]:
import pandas as pd
from utils import *
from statsmodels.tsa.stattools import adfuller

In [17]:
# Loading in econ data and data frame creation

period_dict_ls = [load_econ(fred_daily_id_dict), load_econ(fred_weekly_id_dict), load_econ(fred_monthly_id_dict)]

df = None
for period_dict in period_dict_ls:
    for frame in period_dict.values():
        if df is None:
            df = pd.DataFrame(frame)
        else:
            df = df.merge(pd.DataFrame(frame), on='date', how='outer')

In [18]:
# Data imputation

"""
    Feature start dates:
        WTI 1986-01-02
        Indeed Job Postings 2020-02-01
        CBOE VIX 1990-01-02
        5YR Breakeven Inflation 2003-01-02
        10YR Breakeven Inflation 2003-01-02
        Treasury 10YR minus 2YR 1976-06-01
        Treasury 10YR minus 3M 1982-01-04
        HY OAS 1996-12-31
        Initial Claims SA 1967-01-07
        U3 SA 1948-01-01
        U6 SA 1994-01-01
        PCE SA 1959-01-01
        CPI SA 1947-01-01
        Number Unemployed SA 1948-01-01
        Job Openings SA 2000-12-01
"""

start_date = '2002-12-20'
numeric_cols = [
    'WTI', 'CBOE VIX', 'PCE SA', 'CPI SA', 'U6 SA', 'U3 SA',
    '5YR Breakeven Inflation', '10YR Breakeven Inflation',
    'Treasury 10YR minus 2YR', 'Treasury 10YR minus 3M', 'HY OAS',
    'Initial Claims SA', 'Number Unemployed SA', 'Job Openings SA'
] # Omitted Indeed Jobs data due to lack of data

df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

cs_df, lin_df = impute(df, numeric_cols, start_date)

cs_df = cs_df.loc['2003-01-02':, :].copy()
lin_df = lin_df.loc['2003-01-02':, :].copy()


In [19]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7399 entries, 2003-01-02 to 2025-07-17
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7396 non-null   float64
 1   CBOE VIX                  7398 non-null   float64
 2   PCE SA                    7323 non-null   float64
 3   CPI SA                    7354 non-null   float64
 4   U6 SA                     7354 non-null   float64
 5   U3 SA                     7354 non-null   float64
 6   5YR Breakeven Inflation   7399 non-null   float64
 7   10YR Breakeven Inflation  7399 non-null   float64
 8   Treasury 10YR minus 2YR   7399 non-null   float64
 9   Treasury 10YR minus 3M    7399 non-null   float64
 10  HY OAS                    7398 non-null   float64
 11  Initial Claims SA         7395 non-null   float64
 12  Number Unemployed SA      7354 non-null   float64
 13  Job Openings SA           7323 non-null   flo

In [22]:
lin_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7399 entries, 2003-01-02 to 2025-07-17
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7399 non-null   float64
 1   CBOE VIX                  7399 non-null   float64
 2   PCE SA                    7399 non-null   float64
 3   CPI SA                    7399 non-null   float64
 4   U6 SA                     7399 non-null   float64
 5   U3 SA                     7399 non-null   float64
 6   5YR Breakeven Inflation   7399 non-null   float64
 7   10YR Breakeven Inflation  7399 non-null   float64
 8   Treasury 10YR minus 2YR   7399 non-null   float64
 9   Treasury 10YR minus 3M    7399 non-null   float64
 10  HY OAS                    7399 non-null   float64
 11  Initial Claims SA         7399 non-null   float64
 12  Number Unemployed SA      7399 non-null   float64
 13  Job Openings SA           7399 non-null   flo

In [23]:
# Remove outliers

cs_df = remove_outliers(cs_df, sigma=5)
cs_df = cs_df.dropna()

lin_df = remove_outliers(lin_df, sigma=5)
lin_df = lin_df.dropna()

cs_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7183 entries, 2003-01-02 to 2025-05-01
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7183 non-null   float64
 1   CBOE VIX                  7183 non-null   float64
 2   PCE SA                    7183 non-null   float64
 3   CPI SA                    7183 non-null   float64
 4   U6 SA                     7183 non-null   float64
 5   U3 SA                     7183 non-null   float64
 6   5YR Breakeven Inflation   7183 non-null   float64
 7   10YR Breakeven Inflation  7183 non-null   float64
 8   Treasury 10YR minus 2YR   7183 non-null   float64
 9   Treasury 10YR minus 3M    7183 non-null   float64
 10  HY OAS                    7183 non-null   float64
 11  Initial Claims SA         7183 non-null   float64
 12  Number Unemployed SA      7183 non-null   float64
 13  Job Openings SA           7183 non-null   flo

In [24]:
lin_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7258 entries, 2003-01-02 to 2025-07-17
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7258 non-null   float64
 1   CBOE VIX                  7258 non-null   float64
 2   PCE SA                    7258 non-null   float64
 3   CPI SA                    7258 non-null   float64
 4   U6 SA                     7258 non-null   float64
 5   U3 SA                     7258 non-null   float64
 6   5YR Breakeven Inflation   7258 non-null   float64
 7   10YR Breakeven Inflation  7258 non-null   float64
 8   Treasury 10YR minus 2YR   7258 non-null   float64
 9   Treasury 10YR minus 3M    7258 non-null   float64
 10  HY OAS                    7258 non-null   float64
 11  Initial Claims SA         7258 non-null   float64
 12  Number Unemployed SA      7258 non-null   float64
 13  Job Openings SA           7258 non-null   flo

In [25]:
# Cubic Spline Imputed Master Data Frame Augmented Dickey-Fuller Test Results

for col in numeric_cols:
    test_results = adfuller(cs_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

WTI
p-value:  0.0434
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.9987
------------------------------------
CPI SA
p-value:  0.9988
------------------------------------
U6 SA
p-value:  0.2365
------------------------------------
U3 SA
p-value:  0.1204
------------------------------------
5YR Breakeven Inflation
p-value:  0.0076
------------------------------------
10YR Breakeven Inflation
p-value:  0.005
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.484
------------------------------------
Treasury 10YR minus 3M
p-value:  0.4711
------------------------------------
HY OAS
p-value:  0.0142
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.0908
------------------------------------
Job Openings SA
p-value:  0.8076
------------------------------------


In [26]:
# Linear Imputed Data Frame Augmented Dickey-Fuller Test Results

for col in numeric_cols:
    test_results = adfuller(lin_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

WTI
p-value:  0.051
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.999
------------------------------------
CPI SA
p-value:  0.9988
------------------------------------
U6 SA
p-value:  0.3359
------------------------------------
U3 SA
p-value:  0.2313
------------------------------------
5YR Breakeven Inflation
p-value:  0.011
------------------------------------
10YR Breakeven Inflation
p-value:  0.0042
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.4148
------------------------------------
Treasury 10YR minus 3M
p-value:  0.436
------------------------------------
HY OAS
p-value:  0.021
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.1764
------------------------------------
Job Openings SA
p-value:  0.7616
------------------------------------


In [27]:
# Cubic Spline Imputed Data Frame Augmented Dickey-Fuller Test Results Post Fractional Differencing

fd_cs_df, d = optimal_frac_diff_ffd(cs_df, step=10)

print("Differencing order: ", round(d, 2))
for col in numeric_cols:
    test_results = adfuller(fd_cs_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

Differencing order:  1.0
WTI
p-value:  0.0
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.0
------------------------------------
CPI SA
p-value:  0.0
------------------------------------
U6 SA
p-value:  0.0
------------------------------------
U3 SA
p-value:  0.0
------------------------------------
5YR Breakeven Inflation
p-value:  0.0
------------------------------------
10YR Breakeven Inflation
p-value:  0.0
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.0
------------------------------------
Treasury 10YR minus 3M
p-value:  0.0
------------------------------------
HY OAS
p-value:  0.0
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.0
------------------------------------
Job Openings SA
p-value:  0.0
------------------------------------


In [28]:
fd_cs_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7182 entries, 2003-01-03 to 2025-05-01
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7182 non-null   float64
 1   CBOE VIX                  7182 non-null   float64
 2   PCE SA                    7182 non-null   float64
 3   CPI SA                    7182 non-null   float64
 4   U6 SA                     7182 non-null   float64
 5   U3 SA                     7182 non-null   float64
 6   5YR Breakeven Inflation   7182 non-null   float64
 7   10YR Breakeven Inflation  7182 non-null   float64
 8   Treasury 10YR minus 2YR   7182 non-null   float64
 9   Treasury 10YR minus 3M    7182 non-null   float64
 10  HY OAS                    7182 non-null   float64
 11  Initial Claims SA         7182 non-null   float64
 12  Number Unemployed SA      7182 non-null   float64
 13  Job Openings SA           7182 non-null   flo

In [29]:
# Linear Imputed Data Frame Augmented Dickey-Fuller Test Results Post Fractional Differencing

fd_lin_df, d = optimal_frac_diff_ffd(lin_df, step=10)

print("Differencing order: ", round(d, 2))
for col in numeric_cols:
    test_results = adfuller(fd_lin_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

Differencing order:  1.0
WTI
p-value:  0.0
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.0
------------------------------------
CPI SA
p-value:  0.0
------------------------------------
U6 SA
p-value:  0.0
------------------------------------
U3 SA
p-value:  0.0
------------------------------------
5YR Breakeven Inflation
p-value:  0.0
------------------------------------
10YR Breakeven Inflation
p-value:  0.0
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.0
------------------------------------
Treasury 10YR minus 3M
p-value:  0.0
------------------------------------
HY OAS
p-value:  0.0
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.0
------------------------------------
Job Openings SA
p-value:  0.0
------------------------------------


In [30]:
fd_lin_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7257 entries, 2003-01-03 to 2025-07-17
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WTI                       7257 non-null   float64
 1   CBOE VIX                  7257 non-null   float64
 2   PCE SA                    7257 non-null   float64
 3   CPI SA                    7257 non-null   float64
 4   U6 SA                     7257 non-null   float64
 5   U3 SA                     7257 non-null   float64
 6   5YR Breakeven Inflation   7257 non-null   float64
 7   10YR Breakeven Inflation  7257 non-null   float64
 8   Treasury 10YR minus 2YR   7257 non-null   float64
 9   Treasury 10YR minus 3M    7257 non-null   float64
 10  HY OAS                    7257 non-null   float64
 11  Initial Claims SA         7257 non-null   float64
 12  Number Unemployed SA      7257 non-null   float64
 13  Job Openings SA           7257 non-null   flo

In [31]:
# Cubic Spline Imputed Data Frame Augmented Dickey-Fuller Test Results Post Regular Differencing

pct_cols = ['WTI', 'CBOE VIX', 'PCE SA', 'CPI SA']
diff_cols = [
    '5YR Breakeven Inflation', '10YR Breakeven Inflation',
    'Treasury 10YR minus 2YR', 'Treasury 10YR minus 3M', 'HY OAS',
    'Initial Claims SA', 'Number Unemployed SA', 
    'Job Openings SA', 'U3 SA', 'U6 SA'
]

rd_cs_df = cs_df.copy()
rd_cs_df[diff_cols] = rd_cs_df[diff_cols] - rd_cs_df[diff_cols].shift(1)
rd_cs_df[pct_cols] = rd_cs_df[pct_cols].pct_change()
rd_cs_df = rd_cs_df.dropna()

print("Regular Differencing: ")
for col in numeric_cols:
    test_results = adfuller(rd_cs_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

Regular Differencing: 
WTI
p-value:  0.0
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.0
------------------------------------
CPI SA
p-value:  0.0
------------------------------------
U6 SA
p-value:  0.0
------------------------------------
U3 SA
p-value:  0.0
------------------------------------
5YR Breakeven Inflation
p-value:  0.0
------------------------------------
10YR Breakeven Inflation
p-value:  0.0
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.0
------------------------------------
Treasury 10YR minus 3M
p-value:  0.0
------------------------------------
HY OAS
p-value:  0.0
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.0
------------------------------------
Job Openings SA
p-value:  0.0
------------------------------------


In [32]:
# Linear Imputed Data Frame Augmented Dickey-Fuller Test Results Post Regular Differencing

rd_lin_df = lin_df.copy()
rd_lin_df[diff_cols] = rd_lin_df[diff_cols] - rd_lin_df[diff_cols].shift(1)
rd_lin_df[pct_cols] = rd_lin_df[pct_cols].pct_change()
rd_lin_df = rd_lin_df.dropna()

print("Regular Differencing: ")
for col in numeric_cols:
    test_results = adfuller(rd_lin_df[col])
    print(col)
    print("p-value: ", round(test_results[1],4))
    print("---"*12)

Regular Differencing: 
WTI
p-value:  0.0
------------------------------------
CBOE VIX
p-value:  0.0
------------------------------------
PCE SA
p-value:  0.0
------------------------------------
CPI SA
p-value:  0.0
------------------------------------
U6 SA
p-value:  0.0
------------------------------------
U3 SA
p-value:  0.0
------------------------------------
5YR Breakeven Inflation
p-value:  0.0
------------------------------------
10YR Breakeven Inflation
p-value:  0.0
------------------------------------
Treasury 10YR minus 2YR
p-value:  0.0
------------------------------------
Treasury 10YR minus 3M
p-value:  0.0
------------------------------------
HY OAS
p-value:  0.0
------------------------------------
Initial Claims SA
p-value:  0.0
------------------------------------
Number Unemployed SA
p-value:  0.0
------------------------------------
Job Openings SA
p-value:  0.0
------------------------------------


In [46]:
# Download data frames

fd_cs_df.index.name = 'date'
fd_lin_df.index.name = 'date'

data_dict = {
    "fd_cubic": fd_cs_df,
    "fd_linear": fd_lin_df,
    "rd_cubic": rd_cs_df,
    "rd_linear": rd_lin_df
}

base_path = "/Users/henrycosentino/Desktop/Python/Projects/State Dependent CVaR & VaR/data"

for key, df in data_dict.items():
    filename = f"{base_path}/{key}_processed.csv"
    df.to_csv(filename, index=True)